In [1]:
import pickle
import numpy as np

In [2]:
def load_obj(name ):
    with open('obj/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

def save_obj(obj, name ):
    with open('obj/' + name + '.pkl', 'wb') as f:
        pickle.dump(obj, f)

In [3]:
from classifier.features import FeatureGenerator

In [4]:
lm = load_obj("LanguageModel")

In [5]:
def get_formated_text(text):
    textFormatter = TextFormatter(text)
    words = textFormatter.get_query_list()
    query = textFormatter.text
    
    return query, words

In [6]:
from random import random
from utils import TextFormatter

X = []
y = []
fg = FeatureGenerator(lm)
i = 0
with open("queries_all.txt") as f:
    content = f.readlines()

for line in content:
    if random() < 0.2:
        i += 1
    else:
        continue
        
    if i > 50000:
        break
        
    queries = line.split('\t')
    
    if len(queries) == 2:
        y.append(0)
        query, words = get_formated_text(queries[0])
        X.append(fg.generate_features(query, words))
        
        y.append(1)
        query, words = get_formated_text(queries[1])
        X.append(fg.generate_features(query, words))
        
    else:
        y.append(1)
        query, words = get_formated_text(queries[0])
        X.append(fg.generate_features(query, words))

In [7]:
len(y), len(X)

(52803, 52803)

In [8]:
X = np.asarray(X)
y = np.asarray(y)

In [9]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score, accuracy_score

In [10]:
f1 = []
acc = []
kf = KFold(n_splits=4, shuffle=True)
for train_index, test_index in kf.split(X):
    y_train, y_test = y[train_index], y[test_index]
    X_train, X_test = X[train_index], X[test_index]
    
    gb = GradientBoostingClassifier(n_estimators=200, max_depth=5, loss='exponential')
    gb.fit(X_train, y_train)
    
    y_predicted = gb.predict(X_test)
    f1.append(f1_score(y_test, y_predicted, pos_label=0))
    acc.append(accuracy_score(y_test, y_predicted))
    
print(sum(f1)/len(f1))
print(sum(acc)/len(acc))

0.842428672680037
0.9855500558956736


In [11]:
save_obj(gb, "classifier")
